In [1]:
import pandas as pd
import numpy as np

In [2]:
%store -r BER_Dublin_specific

BER_Dublin_specific.head()

,Postcodes,DwellingTypeDescr,Year_of_Construction,DeliveredEnergyMainWater,MainWaterHeatingFuel,WHMainSystemEff,WHEffAdjFactor,DeliveredEnergySupplementaryWater,SupplWHFuel,DeliveredEnergyMainSpace,...,DeliveredEnergySecondarySpace,HSSupplSystemEff,SupplSHFuel,ElecImmersionInSummer,SolarHeatFraction,HSSupplHeatFraction,CHPSystemType,CHPUnitHeatFraction,gsdHSSupplHeatFraction,TotalFloorArea
2,Co. Dublin,Mid-terrace house,2003,5744.089,Mains Gas,90.3,0.95,0.000,28.0,10826.731,...,3440.495,30.0,8.0,NO,NaN,0.1,NaN,NaN,NaN,131.48
6,Dublin 14,Semi-detached house,1962,4287.963,Mains Gas,72.0,0.95,1041.995,28.0,20046.920,...,2116.064,72.0,2.0,YES,NaN,0.1,NaN,NaN,NaN,95.34
15,Dublin 11,End of terrace house,1972,4087.056,Mains Gas,65.0,0.95,840.445,28.0,14756.551,...,1608.386,63.0,2.0,YES,NaN,0.1,NaN,NaN,NaN,87.76
18,Dublin 12,Mid-terrace house,1939,5879.203,Mains Gas,65.0,0.95,0.000,28.0,11019.516,...,1891.684,40.0,2.0,NO,NaN,0.1,NaN,NaN,NaN,59.31
22,Dublin 18,Top-floor apartment,2004,3348.863,Mains Gas,78.3,0.95,824.336,28.0,8743.726,...,0.000,0.0,1.0,YES,NaN,0.0,NaN,NaN,NaN,74.56


Convert Secondary SH & HW encodings to fuel types
--

In [3]:
## Secondary HW

""" Only one type of supplementary HW system recognised in the BER,
    namely immersion heater located in the HW cylinder, so fuel if specified is always Electricity
    
    Because encoding makes no sense it will be assumed that any non-zero DeliveredEnergySupplementaryWater is electricity
    
    """

def convert_SecHWDemand_to_Fuel (s):
    if s > 0:
        return "Electricity"
    return None

convert_SecHWDemand_to_Fuel = np.vectorize(convert_SecHWDemand_to_Fuel)

# OVERWRITE BER "SupplWHFuel"
BER_Dublin_specific["SupplWHFuel"] = convert_SecHWDemand_to_Fuel(BER_Dublin_specific["DeliveredEnergySupplementaryWater"])

Convert all gsdHSSupplHeatFraction, HSSupplSystemEff into fraction format
--

In [4]:
def convert_to_Fraction(s):
    
    if pd.notnull(s):
        if s > 1:
            return s / 100
    return s


convert_to_Fraction = np.vectorize(convert_to_Fraction)

BER_Dublin_specific["gsdHSSupplHeatFraction_edited"] = convert_to_Fraction(
    BER_Dublin_specific["gsdHSSupplHeatFraction"]
)
BER_Dublin_specific["HSSupplSystemEff_edited"] = convert_to_Fraction(
    BER_Dublin_specific["HSSupplSystemEff"]
)

Set null values to 0
---

In [5]:
BER_Dublin_specific.update(
    BER_Dublin_specific[["CHPUnitHeatFraction", "SolarHeatFraction"]].fillna(0)
)

BER_Dublin_specific[["CHPUnitHeatFraction", "SolarHeatFraction"]].head()

,CHPUnitHeatFraction,SolarHeatFraction
2,0.0,0.0
6,0.0,0.0
15,0.0,0.0
18,0.0,0.0
22,0.0,0.0


Store
---

In [6]:
%store BER_Dublin_specific

Stored 'BER_Dublin_specific' (DataFrame)
